# NGAME-NLG prediction fusion 

In [1]:
#| default_exp 28-oak-training-pipeline-with-multitriplet-loss-and-clustering-and-prediction-fusion

In [2]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [3]:
%load_ext autoreload
%autoreload 2

In [6]:
#| export
import os,torch, torch.multiprocessing as mp, pickle
from xcai.basics import *
from xcai.models.MMM0XX import DBT013

In [7]:
import xclib.evaluation.xc_metrics as xc_metrics

In [8]:
os.environ['WANDB_MODE'] = 'disabled'

In [9]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '6,7'
os.environ['WANDB_PROJECT']='xc-nlg_22-oak-training-pipeline-with-multitriplet-loss-and-clustering'

## Experiment

In [11]:
#| export
block = XCBlock.from_cfg('/home/aiscuser/scratch/datasets', 'data', valid_pct=0.001, tfm='xcnlg', 
                         tokenizer='distilbert-base-uncased', smp_features=[('lbl2data',1,2)])

/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [12]:
#| export
args = XCLearningArguments(
    output_dir='/home/aiscuser/scratch/Projects/xc_nlg/outputs/22-oak-training-pipeline-with-multitriplet-loss-and-clustering-2-14',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    save_strategy="steps",
    evaluation_strategy='steps',
    eval_steps=1000,
    save_steps=1000,
    save_total_limit=5,
    num_train_epochs=50,
    predict_with_representation=True,
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.1,
    learning_rate=2e-4,
    generation_num_beams=10,
    generation_length_penalty=1.5,
    predict_with_generation=True,
    representation_search_type='BRUTEFORCE',
    group_by_cluster=True,
    num_clustering_warmup_epochs=2,
    num_cluster_update_epochs=2,
    num_cluster_size_update_epochs=4,
    clustering_type='EXPO',
    minimum_cluster_size=1,
    maximum_cluster_size=300,
    output_concatenation_weight=1.0,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
)

/opt/conda/envs/ptca/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
mname = f'{args.output_dir}/{os.path.basename(get_best_model(args.output_dir))}'

In [27]:
#| export
mname = f'{args.output_dir}/{os.path.basename(get_best_model(args.output_dir))}'
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*2

model = DBT013.from_pretrained(mname, ig_tok=0, bsz=bsz, tn_targ=1000, margin=0.3, tau=0.1, n_negatives=5, 
                               apply_softmax=True, lw=0.01, tie_word_embeddings=False)

In [51]:
#| export
trie = XCTrie.from_block(block)

  0%|          | 0/312330 [00:00<?, ?it/s]

In [57]:
#| export
train_dset = block.train.dset.sample(n=50_000, seed=50)
metric = PrecRecl(block.n_lbl, train_dset.data.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

learn = XCLearner(
    model=model, 
    args=args,
    trie=trie,
    train_dataset=block.train.dset,
    eval_dataset=train_dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [58]:
#| export
if __name__ == '__main__':
    mp.freeze_support()
    train_pred = learn.predict(train_dset)
    
display_metric(train_pred.metrics)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[2024-05-28 11:24:11,921] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/196 [00:00<?, ?it/s]

node-0:1352351:1352351 [0] NCCL INFO Bootstrap : Using eth0:10.13.42.111<0>
node-0:1352351:1352351 [0] NCCL INFO NET/Plugin : Plugin load (librccl-net.so) returned 2 : librccl-net.so: cannot open shared object file: No such file or directory
node-0:1352351:1352351 [0] NCCL INFO NET/Plugin : No plugin found, using internal implementation
node-0:1352351:1352351 [0] NCCL INFO Kernel version: 5.15.0-1042-azure
RCCL version 2.17.1+hip5.7 HEAD:cbbb3d8+

node-0:1352351:1385403 [1] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

node-0:1352351:1385403 [1] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/transport/net_ib.cc:199 NCCL WARN NET/IB : Unable to open device mlx5_0

node-0:1352351:1385403 [1] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

nod

/home/aiscuser/scratch/Projects/xcai/xcai/losses.py:21: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)
/opt/conda/envs/ptca/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


,P@1,P@1_GEN,P@1_REPR,P@3,P@3_GEN,P@3_REPR,P@5,P@5_GEN,P@5_REPR,P@10,P@10_GEN,P@10_REPR,N@1,N@1_GEN,N@1_REPR,N@3,N@3_GEN,N@3_REPR,N@5,N@5_GEN,N@5_REPR,N@10,N@10_GEN,N@10_REPR,PSP@1,PSP@1_GEN,PSP@1_REPR,PSP@3,PSP@3_GEN,PSP@3_REPR,PSP@5,PSP@5_GEN,PSP@5_REPR,PSP@10,PSP@10_GEN,PSP@10_REPR,PSN@1,PSN@1_GEN,PSN@1_REPR,PSN@3,PSN@3_GEN,PSN@3_REPR,PSN@5,PSN@5_GEN,PSN@5_REPR,PSN@10,PSN@10_GEN,PSN@10_REPR,R@10,R@10_GEN,R@10_REPR,R@100,R@100_GEN,R@100_REPR,R@200,R@200_GEN,R@200_REPR,loss,runtime,samples_per_second,steps_per_second
0,84.8,71.0,92.0,44.25,38.0667,50.2167,29.1,25.34,33.9,15.755,14.07,18.295,84.8,71.0,92.0,83.3821,72.9693,93.3876,83.4693,74.0069,94.2242,84.3574,75.7427,94.9679,76.2503,65.033,82.8373,76.6339,65.4124,87.1923,77.3812,65.9956,89.5593,81.7278,69.5761,92.896,76.2503,65.033,82.8373,80.3799,70.6903,89.2766,81.8046,72.6038,91.2146,83.573,74.5524,92.5706,85.73,81.5224,97.6176,99.3207,81.5999,99.3112,99.4084,81.5999,99.4034,0.0073,207.5618,9.636,0.01


In [59]:
#| export
pred_dir = f'{mname}/predictions/'
os.makedirs(pred_dir, exist_ok=True)
with open(f'{pred_dir}/train_predictions.pkl', 'wb') as file: pickle.dump(train_pred, file)

In [20]:
#| export
test_dset = block.test.dset.sample(n=2000, seed=50)
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

learn = XCLearner(
    model=model, 
    args=args,
    trie=trie,
    train_dataset=block.train.dset,
    eval_dataset=test_dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

In [21]:
#| export
if __name__ == '__main__':
    mp.freeze_support()
    test_pred = learn.predict(block.test.dset)
    
display_metric(test_pred.metrics)

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


,P@1,P@1_GEN,P@1_REPR,P@3,P@3_GEN,P@3_REPR,P@5,P@5_GEN,P@5_REPR,P@10,P@10_GEN,P@10_REPR,N@1,N@1_GEN,N@1_REPR,N@3,N@3_GEN,N@3_REPR,N@5,N@5_GEN,N@5_REPR,N@10,N@10_GEN,N@10_REPR,PSP@1,PSP@1_GEN,PSP@1_REPR,PSP@3,PSP@3_GEN,PSP@3_REPR,PSP@5,PSP@5_GEN,PSP@5_REPR,PSP@10,PSP@10_GEN,PSP@10_REPR,PSN@1,PSN@1_GEN,PSN@1_REPR,PSN@3,PSN@3_GEN,PSN@3_REPR,PSN@5,PSN@5_GEN,PSN@5_REPR,PSN@10,PSN@10_GEN,PSN@10_REPR,R@10,R@10_GEN,R@10_REPR,R@100,R@100_GEN,R@100_REPR,R@200,R@200_GEN,R@200_REPR,loss,runtime,samples_per_second,steps_per_second
0,22.45,15.0,28.85,13.4667,9.0333,18.6333,10.3,6.54,13.94,7.01,4.07,8.725,22.45,15.0,28.85,20.7061,14.1675,27.7934,21.6041,14.4409,28.4831,23.7016,15.4117,30.0675,15.6589,9.3126,23.8955,15.4531,9.2676,24.2329,17.4036,9.6159,25.6114,22.3715,10.9931,28.9783,15.6589,9.3126,23.8955,16.301,9.9223,25.0096,17.8551,10.4698,26.2905,20.3005,11.4135,28.0612,28.114,17.6166,33.97,50.6795,17.6643,50.2855,54.1661,17.6643,53.9083,9.4936,155.6752,12.847,0.019


,P@1,P@1_GEN,P@1_REPR,P@3,P@3_GEN,P@3_REPR,P@5,P@5_GEN,P@5_REPR,P@10,P@10_GEN,P@10_REPR,N@1,N@1_GEN,N@1_REPR,N@3,N@3_GEN,N@3_REPR,N@5,N@5_GEN,N@5_REPR,N@10,N@10_GEN,N@10_REPR,PSP@1,PSP@1_GEN,PSP@1_REPR,PSP@3,PSP@3_GEN,PSP@3_REPR,PSP@5,PSP@5_GEN,PSP@5_REPR,PSP@10,PSP@10_GEN,PSP@10_REPR,PSN@1,PSN@1_GEN,PSN@1_REPR,PSN@3,PSN@3_GEN,PSN@3_REPR,PSN@5,PSN@5_GEN,PSN@5_REPR,PSN@10,PSN@10_GEN,PSN@10_REPR,R@10,R@10_GEN,R@10_REPR,R@100,R@100_GEN,R@100_REPR,R@200,R@200_GEN,R@200_REPR,loss,runtime,samples_per_second,steps_per_second
0,22.45,15.0,28.85,13.4667,9.0333,18.6333,10.3,6.54,13.94,7.01,4.07,8.725,22.45,15.0,28.85,20.7061,14.1675,27.7934,21.6041,14.4409,28.4831,23.7016,15.4117,30.0675,15.6589,9.3126,23.8955,15.4531,9.2676,24.2329,17.4036,9.6159,25.6114,22.3715,10.9931,28.9783,15.6589,9.3126,23.8955,16.301,9.9223,25.0096,17.8551,10.4698,26.2905,20.3005,11.4135,28.0612,28.114,17.6166,33.97,50.6795,17.6643,50.2855,54.1661,17.6643,53.9083,9.4936,155.6752,12.847,0.019


In [ ]:
#| export
pred_dir = f'{mname}/predictions/'
os.makedirs(pred_dir, exist_ok=True)
with open(f'{pred_dir}/test_predictions.pkl', 'wb') as file: pickle.dump(test_pred, file)

In [17]:
pred_dir = f'{mname}/predictions/'

with open(f'{pred_dir}/train_predictions.pkl', 'rb') as f: 
    train_pred = pickle.load(f)
    
with open(f'{pred_dir}/test_predictions.pkl', 'rb') as f: 
    test_pred = pickle.load(f)

In [20]:
gen_pred,targ = get_output_sparse(**test_pred.gen_output, n_lbl=block.n_lbl)
repr_pred,_ = get_output_sparse(**test_pred.repr_output, n_lbl=block.n_lbl)
gen_pred.data = np.exp(gen_pred.data)

In [21]:
train_gen_pred,train_targ = get_output_sparse(**train_pred.gen_output, n_lbl=block.n_lbl)
train_repr_pred,_ = get_output_sparse(**train_pred.repr_output, n_lbl=block.n_lbl)
train_gen_pred.data = np.exp(train_gen_pred.data)

## Fusion

In [75]:
def get_topk_pred(pred, topk=200):
    indices = np.argpartition(pred, -topk, axis=1)[:, -topk:]
    data = np.take_along_axis(pred, indices, axis=1)
    indptr = np.arange(0, data.shape[0]*data.shape[1]+1, topk)
    return sparse.csr_matrix((data.flatten(), indices.flatten(), indptr), shape=pred.shape)


In [52]:
lbl_dl = learn.get_test_dataloader(block.lbl_dset)
lbl_repr = learn.get_representation(lbl_dl)

  0%|          | 0/391 [00:00<?, ?it/s]

In [53]:
lbl_lbl = lbl_repr@lbl_repr.T

In [98]:
gen_predl,train_gen_predl = gen_pred@lbl_lbl,train_gen_pred@lbl_lbl

In [77]:
gen_predl,train_gen_predl = get_topk_pred(gen_predl, topk=200),get_topk_pred(train_gen_predl, topk=200)

In [27]:
prop = xc_metrics.compute_inv_propesity(block.train.dset.data.data_lbl, A=0.5, B=0.4)
fuser = ScoreFusion(prop)

In [28]:
fuser.fit(train_gen_pred, train_repr_pred, train_targ)

In [53]:
pred = fuser.predict(gen_pred, repr_pred, beta=0)

In [54]:
output = {
    'targ_idx': test_pred.gen_output['targ_idx'],
    'targ_ptr': test_pred.gen_output['targ_ptr'],
    'pred_idx': torch.tensor(pred.indices),
    'pred_ptr': torch.tensor([q-p for p,q in zip(pred.indptr, pred.indptr[1:])]),
    'pred_score': torch.tensor(pred.data),
}

In [52]:
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])
m = metric(**output)
display_metric(m, remove_prefix=False)

/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,31.1399,20.7286,15.6277,9.8765,31.1399,30.8007,31.7445,33.5378,24.4896,26.6505,28.803,33.1025,24.4896,26.9292,28.598,30.7434,38.1122,52.0618,55.2864


In [55]:
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])
m = metric(**output)
display_metric(m, remove_prefix=False)

/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,30.4053,20.2255,15.3097,9.7354,30.4053,30.135,31.1323,32.987,24.9653,26.6902,28.7182,32.9481,24.9653,27.0559,28.6494,30.7709,37.6288,51.9256,55.2399
